In [0]:
from pyspark.sql.functions import*
from pyspark.sql.window import *

In [0]:
# Mention your Hive path where the raw file is ingested in the variable
uploaded_parh=""

In [0]:
dbutils.fs.cp(uploaded_path,"/FileStore/tables/Annual_finance_raw",True)

In [0]:
# %fs ls /FileStore/tables/Annual_finance_raw

######1. Rename the "industry_code_ANZSIC" and "industry_name_ANZSIC" with "industry_code" and "industry_name” respectively

In [0]:
df=spark.read.format("csv").option("header",True).option("delimiter",",").option("Inferscheam",True).load("/FileStore/tables/Annual_finance_raw")
df=df.withColumnRenamed("industry_code_ANZSIC","industry_code").withColumnRenamed("industry_name_ANZSIC","industry_name")
df.show(5,truncate=False)

+----+-------------+---------------------------------+------------+-----------------------------------------------+-----+-----------------+
|year|industry_code|industry_name                    |rme_size_grp|variable                                       |value|unit             |
+----+-------------+---------------------------------+------------+-----------------------------------------------+-----+-----------------+
|2011|A            |Agriculture, Forestry and Fishing|a_0         |Activity unit                                  |46134|COUNT            |
|2011|A            |Agriculture, Forestry and Fishing|a_0         |Rolling mean employees                         |0    |COUNT            |
|2011|A            |Agriculture, Forestry and Fishing|a_0         |Salaries and wages paid                        |279  |DOLLARS(millions)|
|2011|A            |Agriculture, Forestry and Fishing|a_0         |Sales, government funding, grants and subsidies|8187 |DOLLARS(millions)|
|2011|A            |

#####
2. Create a new columns called "Amount_in_Dollers" using "unit" column where if the unit value is equal to "DOLLARS(millions)" 
then multiply the "value" column with 10000000 else assign it as 0
3. Create a new columns called "Amount_in_Rupees" using "unit" column where if the unit value is equal to "COUNT" 
then multiply the "value" column with 10000000 else assign it as 0

In [0]:
df_final=df.withColumn("Amount_in_Dollers",when(df.unit=="DOLLARS(millions)",df.value*10000000).when(df.unit!="DOLLARS(millions)",0)).withColumn("Amount_in_Rupees",when(df.unit=="COUNT",df.value*10000000).when(df.unit!="COUNT",0))
df_final.show(5,truncate=False)

+----+-------------+---------------------------------+------------+-----------------------------------------------+-----+-----------------+-----------------+----------------+
|year|industry_code|industry_name                    |rme_size_grp|variable                                       |value|unit             |Amount_in_Dollers|Amount_in_Rupees|
+----+-------------+---------------------------------+------------+-----------------------------------------------+-----+-----------------+-----------------+----------------+
|2011|A            |Agriculture, Forestry and Fishing|a_0         |Activity unit                                  |46134|COUNT            |0.0              |4.6134E11       |
|2011|A            |Agriculture, Forestry and Fishing|a_0         |Rolling mean employees                         |0    |COUNT            |0.0              |0.0             |
|2011|A            |Agriculture, Forestry and Fishing|a_0         |Salaries and wages paid                        |279  |DOLL

####
4. Remove the columns "value","unit","rme_size_grp" from the df and save the new dataframe to a new path

In [0]:
df_final=df_final.drop("value","unit","rme_size_grp")

In [0]:
df_final.write.format("csv").mode("overwrite").option("delimiter","|").option("Header",True).save("/FileStore/tables/Annual_finance")

In [0]:
df_a=spark.read.format("csv").option("delimiter","|").option("Header",True).option("Inferschema",True).load("/FileStore/tables/Annual_finance")
df_a.show(5,truncate=False)

+----+-------------+---------------------------------+-----------------------------------------------+-----------------+----------------+
|year|industry_code|industry_name                    |variable                                       |Amount_in_Dollers|Amount_in_Rupees|
+----+-------------+---------------------------------+-----------------------------------------------+-----------------+----------------+
|2011|A            |Agriculture, Forestry and Fishing|Activity unit                                  |0.0              |4.6134E11       |
|2011|A            |Agriculture, Forestry and Fishing|Rolling mean employees                         |0.0              |0.0             |
|2011|A            |Agriculture, Forestry and Fishing|Salaries and wages paid                        |2.79E9           |0.0             |
|2011|A            |Agriculture, Forestry and Fishing|Sales, government funding, grants and subsidies|8.187E10         |0.0             |
|2011|A            |Agriculture, F

####
5. Use the new path data and print Industry wise max amount in dollers and rupees 

In [0]:
df_max=df_a.groupBy("industry_name","industry_code").agg(max(df_a.Amount_in_Dollers).alias("maxInDollers"),max(df_a.Amount_in_Rupees).alias("maxInRupees"))
df_max.show(truncate=False)

+-----------------------------------------------+-------------+------------+-----------+
|industry_name                                  |industry_code|maxInDollers|maxInRupees|
+-----------------------------------------------+-------------+------------+-----------+
|Construction                                   |E            |7.5961E11   |1.8531E12  |
|Administrative and Support Services            |N            |1.3271E11   |1.2354E12  |
|Agriculture, Forestry and Fishing              |A            |1.90239E12  |1.17051E12 |
|Professional, Scientific and Technical Services|M            |8.8699E11   |1.64619E12 |
|Electricity, Gas, Water and Waste Services     |D            |8.1502E11   |2.0583E11  |
|Education and Training                         |P            |7.668E10    |5.3013E11  |
|All Industries                                 |all          |2.512677E13 |1.903893E13|
|Health Care and Social Assistance              |Q            |2.8812E11   |1.68642E12 |
|Transport, Postal an

####
6. Find the rolling sum of Dollers and Rupees based on years

In [0]:
windowSpec  = Window.partitionBy("year").orderBy("year")
df_roll=df_a.withColumn("rollingSumOfDollersByYear",sum(df_a.Amount_in_Dollers).over(windowSpec))\
    .withColumn("rollingSumOfRupeesByYear",sum(df_a.Amount_in_Rupees).over(windowSpec))
df_roll=df_roll.select("year","rollingSumOfDollersByYear","rollingSumOfRupeesByYear").distinct()
df_roll.show()

+----+-------------------------+------------------------+
|year|rollingSumOfDollersByYear|rollingSumOfRupeesByYear|
+----+-------------------------+------------------------+
|2011|             1.0705055E14|             5.901936E13|
|2012|             1.0976783E14|             5.974098E13|
|2013|             1.0965098E14|             6.019233E13|
|2014|              1.143449E14|             6.183447E13|
|2015|             1.2053435E14|             6.435906E13|
|2016|             1.2556098E14|             6.568677E13|
|2017|             1.3217385E14|             6.754188E13|
|2018|             1.3954568E14|             6.959679E13|
|2019|             1.4864001E14|             7.161339E13|
|2020|             1.5515498E14|             7.208901E13|
|2021|             1.6126405E14|             7.170006E13|
+----+-------------------------+------------------------+



####
7. Identifing the Even rows and finding Rolling Sum based on distinct Indust

In [0]:
df_even=df_a.withColumn("id",row_number().over(Window.orderBy("year","industry_code","industry_name","variable","Amount_in_Dollers","Amount_in_Rupees")))
df_even=df_even.filter((df_even.id)%2==0)
df_even=df_even.withColumn("rollingSumRupeesByIndustry",sum(df_a.Amount_in_Rupees).over(Window.partitionBy("industry_code","industry_Name").orderBy("industry_code","industry_Name"))).withColumn("rollingSumDollersByIndustry",sum(df_a.Amount_in_Dollers).over(Window.partitionBy("industry_code","industry_Name").orderBy("industry_code","industry_Name")))
df_even=df_even.select("industry_name","rollingSumRupeesByIndustry","rollingSumDollersByIndustry").distinct()
df_even.show(truncate=False)

+-----------------------------------------------+--------------------------+---------------------------+
|industry_name                                  |rollingSumRupeesByIndustry|rollingSumDollersByIndustry|
+-----------------------------------------------+--------------------------+---------------------------+
|Agriculture, Forestry and Fishing              |2.143338E13               |3.839772E13                |
|Mining                                         |4.9725E11                 |5.50996E12                 |
|Manufacturing                                  |3.462696E13               |4.910199E13                |
|Electricity, Gas, Water and Waste Services     |1.65303E12                |1.89104E13                 |
|Construction                                   |2.712648E13               |2.364896E13                |
|Wholesale Trade                                |9.61449E12                |3.687439E13                |
|Retail Trade                                   |3.1018

####
8. identifing the max of value of Amount_in_Rupees and display the year and industry name 

In [0]:
max_in_rupees=df_a.agg(max("Amount_in_Rupees")).collect()[0][0]
df_industry=df_a.select("industry_name","year").filter(df_a.Amount_in_Rupees==max_in_rupees)
print(max_in_rupees)
df_industry.show(truncate=False)

19038930000000.0
+--------------+----+
|industry_name |year|
+--------------+----+
|All Industries|2020|
+--------------+----+

